# Mise à jour de la BDD de NFT
Source: [binance.com](https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1)

In [1]:
import os
import time
import numpy as np
from utils import *
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 1. Ouverture du marketplace

- Paramétrage du navigateur distant
- Chargement de la page d'accueil
- Activation des cookies 

In [2]:
%%time
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
browser = webdriver.Remote("http://selenium:4444/wd/hub", DesiredCapabilities.FIREFOX)
browser.get(start_url)
cookies = browser.find_element_by_xpath("//button[contains(text(),'Accept')]")
cookies.click()

CPU times: user 10.8 ms, sys: 5.64 ms, total: 16.4 ms
Wall time: 9.24 s


## 2. Défilement des pages sur le marketplace
 - Choix du nombre de nft à récupérer
 - Chargement des pages contenant les nfts à récuperer 

In [3]:
%%time

MAX_NFT_NB = 25#select_number()
CURRENT_PAGE_NUMBER = int(browser.current_url.split("page=")[-1].split("&")[0])
NFT_NUMBER_PER_PAGE = int(browser.current_url.split("rows=")[-1].split("&")[0])
if MAX_NFT_NB/NFT_NUMBER_PER_PAGE == MAX_NFT_NB//NFT_NUMBER_PER_PAGE:
  scroll_down = np.arange((MAX_NFT_NB//NFT_NUMBER_PER_PAGE)-1)
else:
  scroll_down = np.arange(MAX_NFT_NB//NFT_NUMBER_PER_PAGE)
for i in tqdm(scroll_down, postfix=' Scrolling down on the main page'):
  while CURRENT_PAGE_NUMBER == (browser.current_url.split("page=")[-1]).split("&")[0]:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  CURRENT_PAGE_NUMBER = (browser.current_url.split("page=")[-1]).split("&")[0]

100%|██████████| 1/1 [00:00<00:00, 29.30it/s,  Scrolling down on the main page]

CPU times: user 36.7 ms, sys: 17.6 ms, total: 54.3 ms
Wall time: 161 ms


## 3. Récupération des liens des pages détaillées
- Détection des boutons donnant accès aux infos détaillées

In [4]:
%%time

detailed_pages = []
pbar = tqdm(total = MAX_NFT_NB, postfix=" Searching of NFT detailed pages")
BAR_LEVEL = 0
while len(detailed_pages) < MAX_NFT_NB:
  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  detailed_pages = browser.find_elements_by_xpath('//button[contains(text(),"BSC")]')
  pbar.update(max(0,len(detailed_pages)-BAR_LEVEL))
  BAR_LEVEL = len(detailed_pages)
pbar.close()

32it [00:02, 12.40it/s,  Searching of NFT detailed pages]                         

CPU times: user 148 ms, sys: 38.2 ms, total: 186 ms
Wall time: 2.59 s


## 4. Parcours des pages détaillés
- Récupération des données utiles sur les nft
- Nettoyage des chaînes de caractères
- Stockage des dictionnaires de données
- Fermeture du navigateur distant

In [5]:
%%time
rm_old_save=True
save_file = f'binance_{datetime.now()}.txt'.replace(":","")

for page in tqdm(detailed_pages, postfix=" Scraping of detailed NFT pages"):
  page.click()
  browser.switch_to.window(browser.window_handles[-1])
  add_to_json(parse_nft(browser), save_file)
  browser.close()
  browser.switch_to.window(browser.window_handles[-1])

if rm_old_save:
    [os.remove(file) for file in os.listdir('.') if file.find('.json')!=-1]    
os.rename(save_file, save_file.replace(".txt",".json"))
browser.quit()

100%|██████████| 32/32 [01:49<00:00,  3.43s/it,  Scraping of detailed NFT pages]


CPU times: user 1.98 s, sys: 589 ms, total: 2.57 s
Wall time: 1min 50s


___

In [6]:
# Process the data
def process_data(nft):
    del nft['id']# = str(hash(nft['contract_address']))
    nft['date'] = str(datetime.now())
    amount, currency = nft['currency'].split(' ')
    nft['amount'] = float(amount)
    nft['currency'] = currency
    return nft

In [7]:
# Necessary packages
import json
import requests

# Fetch the data
filename = save_file.replace("txt", "json")
with open(filename, 'r') as f:
    data = json.load(f)

# Apply processing
nfts = [*map(process_data, data)]

In [8]:
nfts[0]

{'name': 'Floyd Mayweather',
 'image': 'https://public.nftstatic.com/static/nft/zipped/c1a617b0ba8e44f78f6ba82aab9b4224_zipped.jpeg',
 'amount': 0.3,
 'currency': 'ETH',
 'creator': 'NFTshka',
 'date': '2021-09-23 10:11:31.727693',
 'description': 'Author Krenz137',
 'contract_address': '0x1dDB2C0897daF18632662E71fdD2dbDC0eB3a9Ec',
 'link': 'https://www.binance.com/en/nft/goods/detail?productId=7621124&isProduct=1'}

In [12]:
success = 0
failure = 0
total = len(nfts)

for nft in nfts:
    url = 'http://api:8000/add_nft'
    headers = {'accept': 'application/json', 'Content-Type': 'application/json'}
    res = requests.post(url, data=json.dumps(nft), headers=headers)
    if res.status_code == 201:
        success += 1
    else:
        failure += 1
    print(f"Success={success}/{total}; Failure={failure}/{total} - {res.status_code}", end="\r")

In [10]:
res.text

'{"date":"2021-09-23 10:11:36.897236","status":"Success"}'

In [11]:
requests.get('http://api:8000/nfts').text

'[{"_id":"614b4e10ea9d15827ee632ed","name":"Jejune Hustler","image":"https://lh3.googleusercontent.com/Hu8R9ZDsElNZ_S3bnbUGIY3vS6h3wjzm15d7ZOjoMJp17omUKtY3h78syRh2jaVQUiA34Ml0O2881vHlHtymipIpGz0_jcNCMTDuWA=w600","amount":0.0001,"currency":"ETH","creator":"Neon District Season One Item","date":"March 17, 2022 at 1:46pm CEST","description":"Armor found within Neon District. \\nA Neon District: Season One game item, playable on https://portal.neondistrict.io.\\nNeon District is a free-to-play cyberpunk role-playing game. Collect characters and gear, craft and level up teams, and battle against other players through competitive multiplayer and in turn-based combat.","contract_address":"0x7227e371540cf7b8e512544ba6871472031f3335","link":"https://opensea.io/assets/matic/0x7227e371540cf7b8e512544ba6871472031f3335/158456332462567099841992013138"},{"_id":"614b4ed4ea9d15827ee632f2","name":"Abstract","image":"https://public.nftstatic.com/static/nft/zipped/84f8bfb7bde64a3298f57dc3bb1c05d9_zipped.p